### Descripción del programa
Este programa sirve para el cálculo de columnas de concreto reforzado siguiendo los lineamientos de la Norma Colombiana de Construcción Sismoresistente <b>(NSR-10)</b>.

La interfaz gráfica permite ingresar los valores de diseño y ver los resultados de forma interactiva. Los resultados consisten en las curvas de interacción de la columna para cuantías del 1%, 2%, 3% y 4%, así como la curva de interacción correspondiente a una cuantía de tentativa de diseño (línea punteada). La estrella en el gráfico muestra el punto correspondiente a la carga axial última y el momento flector último.

#### ¿Cómo usar este programa? 
Ejecute la celda que contiene el código (shift + enter). Una vez se ejecuta la celda, se crean las curvas de interacción y los controles interactivos. Modifique los datos a través de los controles interactivos. No modifique el código de la celda; no es necesario para usar el programa.

In [1]:
import ipywidgets as wdg
import matplotlib.pyplot as plt

# Definición de la función principal
def columnas_concreto(h, b, r, fc, fy, rho, Pu, Mu):   
 
    # h: Ancho de la sección (En m)
    # b: Profundidad de la sección (En m)
    # r: Recubrimiento (En m)
    # fy: Esfuerzo de fluencia del acero En MPa. 
    
    beta = 0.85                                    # Ojo si f'c > 5000 psi
    d = h - r
    n = 100                                        # Cantidad de puntos
    c = [i*(5*h - r)/n + r for i in range(n+1)]    # Distancia al eje neutro
    a = []                                         # Longitud del bloque de compresión equivalente
    for i in range(len(c)):
        if c[i] > h/beta:
            a.append(h)
        else:
            a.append(beta*c[i])

    cuantias = [0.01, 0.02, 0.03, 0.04]
    E = 210000.0                                   # Módulo de elasticidad del acero (En MPa)
    epsilon_c = -0.003

    # Compatibilidad de deformaciones
    def_compresion = [(1 - r/i)*epsilon_c for i in c]
    def_traccion = [(h/i - 1 - r/i)*(-epsilon_c) for i in c]

    # Esfuerzo en el acero a compresion
    esf_compresion = []
    for i in range(len(def_compresion)):
        if abs(def_compresion[i]) <= (fy/E):
            esf_compresion.append(E*def_compresion[i])
        else:
            esf_compresion.append(-fy)

    # Esfuerzo en el acero a tracción
    esf_traccion = []
    for i in range(len(def_traccion)):
        if def_traccion[i] <= (-fy/E):
            esf_traccion.append(-fy)
        elif -fy/E < def_traccion[i] < fy/E:
            esf_traccion.append(E*def_traccion[i])
        else:
            esf_traccion.append(fy)

    # Definición de funciones importantes *****************
    def Axial_nom(cu):
        Pn = []
        for i in range(len(c)):
            Pn.append(0.85*fc*a[i]*b - esf_compresion[i]*(cu*h*b/2) - esf_traccion[i]*(cu*h*b/2))
        return Pn  

    def Mom_nom(cu):
        Mn = []
        for i in range(len(c)):
            Mn.append(0.85*fc*a[i]*b*(h/2 - a[i]/2) + (-1)*esf_compresion[i]*(cu*h*b/2)*(h/2 - r) + esf_traccion[i]*(cu*h*b/2)*(d - h/2))
        return Mn

    def cargas_ultimas(Pn, Mn):
        Pu = [] ; Mu = []
        for i in range(len(c)):
            if Pn[i] >= Pn[Mn.index(max(Mn))]:
                Pu.append(0.65*Pn[i])                        # 0.65 para columnas con estribos; 0.75 para col. zunchadas
                Mu.append(0.65*Mn[i])
            else:
                if def_traccion[i] < 0.002:
                    Pu.append(0.65*Pn[i])
                    Mu.append(0.65*Mn[i])
                elif (0.002 <= def_traccion[i] <= 0.005):
                    Pu.append((0.65 + (250/3)*(def_traccion[i] - 0.002))*Pn[i])
                    Mu.append((0.65 + (250/3)*(def_traccion[i] - 0.002))*Mn[i])
                else:
                    Pu.append(0.90*Pn[i])
                    Mu.append(0.90*Mn[i])
        return Pu, Mu
    # *****************************************************
    
    P, M = cargas_ultimas(Axial_nom(rho), Mom_nom(rho))
    Pu1, Mu1 = cargas_ultimas(Axial_nom(cuantias[0]), Mom_nom(cuantias[0]))
    Pu2, Mu2 = cargas_ultimas(Axial_nom(cuantias[1]), Mom_nom(cuantias[1]))
    Pu3, Mu3 = cargas_ultimas(Axial_nom(cuantias[2]), Mom_nom(cuantias[2]))
    Pu4, Mu4 = cargas_ultimas(Axial_nom(cuantias[3]), Mom_nom(cuantias[3]))
    
    # Representación gráfica de los resultados
    plt.figure(figsize=(10, 7))
    plt.plot(Mu1, Pu1, label='1%')
    plt.plot(Mu2, Pu2, label='2%')
    plt.plot(Mu3, Pu3, label='3%')
    plt.plot(Mu4, Pu4, label='4%')
    plt.plot(M, P, 'r--', label='Cuantía de diseño')
    plt.scatter(Mu, Pu, color='purple', marker='*', s=200)
    plt.legend()
    plt.xlim(0, 1.05*max(Mu4))
    plt.ylim(0, 1.05*max(Pu4))
    plt.grid()
    plt.title('Curvas de interacción NSR-10', size=16)
    plt.xlabel('Momento Último (MN)', size=14)
    plt.ylabel('Carga Axial Última (MN-m)', size=14)
    plt.show;
    # Aquí acaba la función principal **************
    

# Interfaz Gráfica de usuario
peralte = wdg.FloatText(description='Peralte (m)', step=0.01, value=0.3)
ancho = wdg.FloatText(description='Ancho (m)', step=0.01, value=0.3)
recubrimiento = wdg.FloatText(description='Recubrimiento (m)', step=0.005, value=0.03)
concreto = wdg.FloatText(description="f'c (MPa)", step=0.5, value=21)
acero = wdg.FloatText(description='fy (MPa)', step=1, value=420)
cuantia = wdg.FloatText(description='Cuantía', step=0.001, value=0.015)
P_ultimo = wdg.FloatText(description='P ultima', step=0.1, value=0.75)
M_ultimo = wdg.FloatText(description='M ultimo', step=0.01, value=0.125)

asignaciones = {
    'h': peralte,
    'b': ancho,
    'r': recubrimiento,
    'fc': concreto,
    'fy': acero,
    'rho': cuantia,
    'Pu': P_ultimo,
    'Mu': M_ultimo,
}

h1 = wdg.HBox([peralte, ancho, recubrimiento])
h2 = wdg.HBox([concreto, acero, cuantia])
h3 = wdg.HBox([P_ultimo, M_ultimo])
v = wdg.VBox([h1, h2, h3])

GUI = wdg.interactive_output(columnas_concreto, asignaciones)
display(GUI, v)

Output()